### Armar un sistema de recomendación de películas. 

Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas. 

Se ordenarán según el score de similaridad y devolverá:
- una lista de Python con 5 valores, 
- cada uno siendo el string del nombre de las películas con mayor puntaje 
- en orden descendente.

In [21]:
import pandas as pd
from datetime import datetime

In [48]:
df = pd.read_csv('merged_movies_credits.csv')

In [44]:
df2 = pd.read_csv('ML_dataset.csv')

In [49]:
df.columns

Index(['budget', 'genres', 'id', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_day', 'cast',
       'director_name', 'crew'],
      dtype='object')

In [45]:
df2.columns

Index(['id', 'title', 'genres', 'original_language', 'spoken_languages',
       'popularity', 'production_companies', 'production_countries',
       'release_year', 'vote_count', 'vote_average'],
      dtype='object')

In [46]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45344 entries, 0 to 45343
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45344 non-null  int64  
 1   title                 45344 non-null  object 
 2   genres                45344 non-null  object 
 3   original_language     45344 non-null  object 
 4   spoken_languages      45344 non-null  object 
 5   popularity            45344 non-null  float64
 6   production_companies  45344 non-null  object 
 7   production_countries  45344 non-null  object 
 8   release_year          45344 non-null  int64  
 9   vote_count            45344 non-null  int64  
 10  vote_average          45344 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 3.8+ MB


In [67]:
# podemos utilizar técnicas de procesamiento de lenguaje natural (NLP) y medidas de similitud 
# para encontrar películas similares.

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def recomendacion(titulo):
    
    # Crear un vectorizador TF-IDF para convertir las sinopsis de las películas en vectores numéricos
    tfidf = TfidfVectorizer(stop_words='english')
    df2['overview'] = df2['overview'].fillna('')  # Asegurarse de que no haya valores nulos en la columna 'overview'
    tfidf_matrix = tfidf.fit_transform(df2['overview'])
    
    # Calcular las similitudes de coseno entre todas las películas
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # Obtener el índice de la película que coincide con el título ingresado
    indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()
    idx = indices[titulo]
    
    # Calcular las similitudes de coseno entre la película seleccionada y todas las demás películas
    sim_scores = list(enumerate(cosine_similarities[idx]))
    
    # Ordenar las películas según su similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de las 5 películas más similares (excluyendo la película seleccionada)
    similar_indices = [i[0] for i in sim_scores[1:6]]
    
    # Obtener los títulos de las películas más similares
    similar_movies = df2['title'].iloc[similar_indices]
    
    return similar_movies


In [ ]:
recomendacion('Toy Story')

In [60]:
# Crear una instancia de OneHotEncoder y especificar los géneros más relevantes
top_genres = ['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy', 'Romance', 'Sci-Fi', 'Thriller', 'Family', 'Animation']
#encoder = OneHotEncoder(categories=[top_genres])

In [62]:
#encoder

OneHotEncoder(categories=[['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy',
                           'Romance', 'Sci-Fi', 'Thriller', 'Family',
                           'Animation']])

In [58]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

def recomendacion_KNN(titulo):
       
    # Obtener el índice de la película que coincide con el título ingresado
    indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()
    idx = indices[titulo]
    
    # Seleccionar las características relevantes para el algoritmo KNN
    features = ['popularity', 'vote_count','vote_average','release_year']
    X = df2[features]

        
       
    # Normalizar las características
    X = (X - X.mean()) / X.std()
    
    # Crear una instancia del algoritmo KNN
    knn = NearestNeighbors(n_neighbors=6)  # Consideramos 6 vecinos, incluyendo la película seleccionada
    
    # Entrenar el modelo KNN
    knn.fit(X)
    
    # Obtener la distancia y los índices de los vecinos más cercanos
    distances, indices = knn.kneighbors(X.iloc[idx].values.reshape(1, -1))
    
    # Obtener los títulos de las películas más similares (excluyendo la película seleccionada)
    similar_movies = df2['title'].iloc[indices[0][1:6]]
    
    return similar_movies


In [59]:
recomendacion_KNN('Toy Story')

c:\Users\TheFercho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


359                                  The Lion King
3453                                     Gladiator
8228                               The Incredibles
1916                           Saving Private Ryan
11919    Harry Potter and the Order of the Phoenix
Name: title, dtype: object